In [16]:
import sys
sys.path.append("../../models/PFN")
from dataset_pfn import PFNDataset, processed2tau
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from augmented_pfn_model import AugParticleFlowNetwork as Model
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
import os, json
from scipy import interpolate
%env HDF5_USE_FILE_LOCKING=FALSE

env: HDF5_USE_FILE_LOCKING=FALSE


In [2]:
test_path = "../../datasets/test.h5"

#Loading testing dataset
test_set = PFNDataset(test_path, preprocessed=True)
testloader = DataLoader(test_set, shuffle=True, batch_size=500, num_workers=30, pin_memory=True, persistent_workers=True)

/u/avroy/.conda/envs/toptagger_env/lib/python3.9/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
all_models = [f for f in os.listdir("../../models/PFN/trained_models") if "_best" in f and "_aug" in f]
print("\n".join(all_models))

PFN_best_v20_augmented
PFN_best_baseline_augmented
PFN_best_v9_augmented
PFN_best_v40_augmented
PFN_best_v30_augmented


In [4]:
def eval2(model):
    labels = []
    preds = []
    with torch.no_grad():
        for x,m,y,_ in tqdm(testloader):
            x = x.cuda()
            m = m.cuda()
            pred = model(x, m)
            labels.append(y[:,1].cpu().numpy())
            preds.append(pred[:,1].cpu().numpy())
    labels = np.concatenate(labels, axis=None)
    preds = np.concatenate(preds, axis=None)
    return labels, preds

In [13]:
def evalFC(model, data, mask_indices = [-1], mask_values = [0.]):
    lfs = data.cuda()
    for mask_index, mask_value in zip(mask_indices, mask_values):
        if mask_index >= 0:
            lfs[:, mask_index] = mask_value*torch.ones(lfs[:, mask_index].shape)  
    for ii, layer in enumerate(FcLayers):
        if ii == len(FcLayers)-1: 
            res = layer.forward(lfs).cpu()
        elif ii == len(FcLayers)-2:
            lfs = layer.forward(lfs)
            psmax = lfs.cpu()
        else:
            lfs = layer.forward(lfs)
    return psmax, res

def evalPHI(model, test_set, d_mean = [], mask_indices = []):
    myDL = DataLoader(test_set, batch_size=1024)
    with torch.no_grad():
        for idx, (d, m, l, ad) in enumerate(myDL):
            # for mask_index in mask_indices:
            #     try:
            #         d[:, mask_index[0], mask_index[1]] = d_mean[mask_index[0], mask_index[1]]
            #     except:
            #         d[:, mask_index[0], mask_index[1]] = d_mean[mask_index[1]]
            d2 = torch.flatten(d, start_dim=0, end_dim=1)
            lfs = model._modules['phi'].forward(d2.cuda())
            lfs = torch.stack(torch.split(lfs.permute(1, 0), 200, dim=1), 0)
            lfs = lfs * m.cuda().bool().float()
            lfs = lfs.sum(-1)
            this_lfs = lfs.cpu()
            this_taus = processed2tau(d,ad)
            psmax, res = evalFC(model, 
                                torch.cat((lfs, this_taus.cuda()[:,:7]),1).float())
            # res = model._modules['fc'].forward(lfs)
            if idx == 0:
                latents = this_lfs.cpu()
                labels = l.cpu()
                presoftmax = psmax.cpu()
                preds = res.cpu()
                aug_data = ad.cpu()
                taus = this_taus.cpu()
            else:
                latents = torch.cat((latents, this_lfs.cpu()), 0)
                labels = torch.cat((labels, l.cpu()), 0)
                presoftmax  = torch.cat((presoftmax, psmax.cpu()), 0)
                preds  = torch.cat((preds, res.cpu()), 0)
                aug_data = torch.cat((aug_data, ad.cpu()), 0)
                taus = torch.cat((taus, this_taus.cpu()), 0)
            if idx == 100:
                break
    return latents, labels, presoftmax, preds, aug_data, taus


In [17]:
#loading model
for modelname in all_models:
    model_dict = json.load(open("../../models/PFN/trained_model_dicts/" + modelname.replace("_best","") + ".json"))
    #print(modelname, model_dict)
    label = model_dict['label']
    f_nodes = list(map(int, model_dict['f_nodes'].split(',')))
    phi_nodes = list(map(int, model_dict['phi_nodes'].split(',')))

    model = Model(input_dims=3, Phi_sizes=phi_nodes, F_sizes=f_nodes).cuda()
    model.load_state_dict(torch.load("../../models/PFN/trained_models/" + modelname ))
    nparams = sum(p.numel() for p in model.parameters())
    
    FcLayers = []
    PhiLayers = []
    for layer in model._modules['fc']:
        #print(layer)
        FcLayers.append(layer)
    #print('\n')
    for layer in model._modules['phi']:
        #print(layer)
        PhiLayers.append(layer)
    
    _, labels, _, preds, _, _ = evalPHI(model, test_set, d_mean = [], mask_indices = [])
    labels = labels[:,0]
    preds = preds[:,0]
    #labels, preds = eval2(model)
    accuracy = accuracy_score(labels, preds.round())*100
    
    auc = roc_auc_score(labels, preds)*100
    
    fpr, tpr, thresholds = roc_curve(labels, preds, drop_intermediate=False)
    intrp = interpolate.interp1d(fpr, tpr)
    brr = 1./(1- intrp(0.7))
    print("{} \t\t Params: {}\t ROC-AUC: {:.2f}%, Accuracy: {:.2f}%, BRR: {:.2f}".format(modelname,nparams,auc,accuracy,brr))
    #print(modelname, "\t", "ROC-AUC: {:.4f}% Accuracy: {:.4f}%".format(auc, accuracy))

[100, 64, 64] [64, 32]


/tmp/ipykernel_3480776/155672293.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  brr = 1./(1- intrp(0.7))


PFN_best_v20_augmented 		 Params: 17778	 ROC-AUC: 99.99%, Accuracy: 99.93%, BRR: inf
[100, 100, 256] [100, 100, 100]


/tmp/ipykernel_3480776/155672293.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  brr = 1./(1- intrp(0.7))


PFN_best_baseline_augmented 		 Params: 83158	 ROC-AUC: 99.99%, Accuracy: 99.93%, BRR: inf
[100, 100, 64] [64, 64]
PFN_best_v9_augmented 		 Params: 25862	 ROC-AUC: 99.99%, Accuracy: 99.94%, BRR: 26152.00
[100, 100, 64] [100, 100, 100]


/tmp/ipykernel_3480776/155672293.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  brr = 1./(1- intrp(0.7))


PFN_best_v40_augmented 		 Params: 44566	 ROC-AUC: 100.00%, Accuracy: 99.94%, BRR: inf
[100, 100, 128] [100, 100, 100]
PFN_best_v30_augmented 		 Params: 57430	 ROC-AUC: 99.99%, Accuracy: 99.94%, BRR: 52304.00
